_All data was collected on 05 October 2024._

In [1]:
import hashlib
import logging
import os
import shutil
import sqlite3
import time
import random
import re
import requests
from collections import defaultdict

import networkx as nx
import praw
import pandas as pd
from selenium import webdriver #also need a standalone chromedriver executable
from tqdm.notebook import tqdm, trange
from dotenv import load_dotenv
from matplotlib_venn import venn2
from pprint import pprint

In [2]:
logging.basicConfig(level=logging.INFO)
load_dotenv(); #loading environmental variables from .env

In [3]:
def notify(title, message):
    #Send a Telegram message using noti
    #https://github.com/variadico/noti
    if shutil.which("noti"):
        os.system("noti -g -t '%s' -m '%s'" % (title, message))

# Getting list of subreddits

There are two Reddit lists that list best/popular communities:
* www.reddit.com/best/communities/1/
* www.reddit.com/subreddits/

The first one is endless and doesn't include many known popular communities. The reason is probably that Reddit doesn't want to suggest NSFW or controversial communities to new users or advertisers. Some examples of missing subs:
* `r/iamatotalpieceofshit`
* `r/therewasanattempt`
* `r/illusions`
* `r/Palestine`
* `r/PublicFreakout`
* `r/CombatFootage`
* `r/FUCKYOUINPARTICULAR`
* `r/IsItBullshit`
* `r/IdiotsFightingThings`
* `r/JordanPeterson`
* `r/MensRights`
* `r/OneSecondBeforeDisast`
* `r/TIHI`
* `r/ThatsInsane`
* `r/TrueUnpopularOpinion`
* `r/Wellthatsucks`
* `r/Whatcouldgowrong`
* `r/cringe`
* `r/sex`
* `r/stupidquestions`

Therefore, I decided to use www.reddit.com/subreddits/ only. It's a list of 4387 subreddits.

##  Getting most popular subreddits from www.reddit.com/subreddits/

In [ ]:
#Old Reddit blocks non-browser requests, so I've got to use Selenium
pop_subs = set()
with open("popular-subreddits.txt", "w") as out:
    url = "https://www.reddit.com/subreddits"
    regex_subs = re.compile(r'class="titlerow"><a href="https://www.reddit.com/r/([^"]+)/"')
    regex_next = re.compile(r'<span class="next-button">\s*<a href="([^"]*)')
    driver = webdriver.Chrome()

    while True:
        attempts = 0
        matches = None
        while not matches and attempts < 5:
            driver.get(url)
            source = driver.page_source
            next_subs = regex_subs.findall(source)
            matches = regex_next.findall(source)
            time.sleep(5 + 5 * attempts)
            attempts += 1

        if next_subs:
            next_subs = [s for s in next_subs if s] #removing empty strings
            pop_subs |= set(next_subs)
        
        print(f"Processed {url}, {len(next_subs)} subs")
        if not matches:
            break #reached end of list

        [url] = matches
        out.flush()
        time.sleep(5)

print(f"Compiled popular-communities.txt ({len(pop_subs)} subs)")
notify("reddit-mods-ds", f"Compiled popular-communities.txt ({len(pop_subs)} subs)")

In [37]:
len(pop_subs)

4386

In [25]:
with open('popular-subreddits.txt', 'w') as f:
    f.write('\n'.join())

# Inspecting subreddits

To increase speed, I will be using two different Reddit accounts. I have authorized a `script` application for each of them on www.reddit.com/prefs/apps.

I will be using the schema
![](schema.png)

In [20]:
class DBFactory:

    db = None

    def __call__(self):
        # DBFactory.db = DBFactory.db or sqlite3.connect('db.sqlite3')
        DBFactory.db = sqlite3.connect('db-new.sqlite3')
        self.create_tables()
        self.init_tables()
        return self.db

    
    def create_tables(self):
        cursor = self.db.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS subs(
                name TEXT PRIMARY KEY,
                nsubscr INTEGER,
                processed BOOLEAN DEFAULT 0
            )
        ''')
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS mods(
                mod TEXT,
                sub TEXT,
                PRIMARY KEY (mod, sub)
            )
        ''')
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS users(
                name TEXT PRIMARY KEY,
                is_bot BOOLEAN,
                last_active INTEGER,
                processed BOOLEAN DEFAULT 0
            )
        ''')
        self.db.commit()
        cursor.close()

    def init_tables(self):
        cursor = self.db.cursor()
        self.db.commit()
        cursor.close()
        

In [7]:
class SubInspector:
    
    def __init__(self, reddit):
        self.reddit = reddit
        self.db = DBFactory()()
        self.cursor = self.db.cursor()

    
    def inspect(self, *, sub_name=None):        
        self.cursor.execute("SELECT name FROM subs WHERE name=? AND processed=1", (sub_name,))
        if self.cursor.fetchone():
            return 0 #sub already processed
            
        sub = self.reddit.subreddit(sub_name)
        self.cursor.execute("INSERT OR IGNORE INTO subs(name, nsubscr) VALUES(?,?)", (sub.display_name, sub.subscribers))
        for mod in sub.moderator():
            self.cursor.execute("INSERT OR IGNORE INTO mods(mod, sub) VALUES(?,?)", (mod.name, sub.display_name))
            self.cursor.execute("INSERT OR IGNORE INTO users(name) VALUES(?)", (mod.name,))

        self.cursor.execute("UPDATE subs SET processed = 1 WHERE name = ?", (sub.display_name,))
        self.db.commit()

        logging.info(f'Inspected sub {sub.display_name}')
        return 1 #sub is new


    def set_reddit(self, reddit):
        self.reddit = reddit

    
    def __del__(self):
        self.cursor.close()
            

In [31]:
reddit1 = praw.Reddit(
    client_id=os.getenv("CLIENT_ID_1"),
    client_secret=os.getenv("CLIENT_SECRET_1"),
    user_agent=os.getenv("USER_AGENT_1"),
    username=os.getenv("USER_NAME_1"),
    password=os.getenv("USER_PASS_1")
)

reddit2 = praw.Reddit(
    client_id=os.getenv("CLIENT_ID_2"),
    client_secret=os.getenv("CLIENT_SECRET_2"),
    user_agent=os.getenv("USER_AGENT_2"),
    username=os.getenv("USER_NAME_2"),
    password=os.getenv("USER_PASS_2")
)

In [36]:
with open('popular-subreddits.txt', 'r') as f:
    pop_subs = [s.strip() for s in f.readlines()]

inspector = SubInspector(reddit1)
npr = 0

for i, sub_name in enumerate(tqdm(pop_subs)):    
    if inspector.inspect(sub_name=sub_name):
        npr += 1
        time.sleep(0.5)

    #magic to overcome Reddit's ratelimits
    if npr and npr % 25 == 0:
        inspector.set_reddit(reddit2 if inspector.reddit is reddit1 else reddit1)
    if npr and npr % 50 == 0:
        time.sleep(10)
    if npr and npr % 100 == 0:
        notify("reddit-mod-db", f"Processed {npr} subs, {len(pop_subs) - i} left...") 
    if npr and npr % 500 == 0:
        time.sleep(10)
    if npr and npr % 1000 == 0:
        time.sleep(10)

notify("reddit-mod-db", f"Done processing popular {len(pop_subs)} subs!") 

  0%|          | 0/4386 [00:00<?, ?it/s]

# Writing output 

## Pooling data

In [29]:
db = DBFactory()()
cursor = db.cursor()

In [44]:
with open('../subreddits.csv', 'w') as f:
    f.write('name,n_members\n')
    cursor.execute("""
        SELECT name, nsubscr FROM subs ORDER BY nsubscr DESC;
    """)
    for name, n_members in tqdm(cursor.fetchall()):
        f.write(f'{name},{n_members}\n')

  0%|          | 0/25834 [00:00<?, ?it/s]

In [47]:
with open('../moderators.csv', 'w') as f:
    f.write(',moderator,subreddit\n')
    cursor.execute("""
        SELECT mods.mod, mods.sub FROM mods LEFT JOIN subs ON mods.sub = subs.name ORDER BY subs.nsubscr DESC;
    """)
    for i, (moderator, subreddit) in enumerate(tqdm(cursor.fetchall()), 1):
        f.write(f'{i},{moderator},{subreddit}\n')

  0%|          | 0/125812 [00:00<?, ?it/s]

In [69]:
with open('../bots.csv', 'w') as f:
    f.write('name\n')
    cursor.execute("""
        SELECT name FROM bots;
    """)
    for (name, ) in cursor.fetchall():
        f.write(name + '\n')

## Anonymising personal data

Usernames are personal data, and can potentially be used to identify the individual. We will encode all moderator usernames by hashing them with SHA256.

In [2]:
df_mods = pd.read_csv('../moderators.csv', index_col=0)

In [10]:
sha256_enc = lambda x: hashlib.sha256(x.encode()).hexdigest()
df_mods['moderator'] = df_mods['moderator'].apply(sha256_enc)                         

In [11]:
df_mods.head()

,moderator,subreddit
1,1039a46c82d342ef6c1d1b77cc7175a4cabd8cfb3f2e8d...,announcements
2,f58a99ef5e6677314d0fb1e5e7a0ee7ed8aad22cdf93a3...,announcements
3,c750264db0d83c6a5d722207514ab117a3fce88d9e5a34...,announcements
4,a39672e330f0293d47e69ea37cf54ece0edf739892ed59...,announcements
5,8e1519f03793fb5db6129addf6ef138056d15791508344...,announcements


In [13]:
df_mods.to_csv('../moderators.csv')

## Writing graph 

In [14]:
df_subs = pd.read_csv('../subreddits.csv')
df_mods = pd.read_csv('../moderators.csv', index_col=0)

Writing as a bipartite graph, where both mods and subs are nodes.

In [15]:
n_members_dict = df_subs.set_index('name').to_dict()['n_members']

In [16]:
g = nx.Graph()
g.add_nodes_from(df_mods['moderator'].unique(), bipartite=0)
for sub in df_mods['subreddit'].unique():
    g.add_node(sub, size=n_members_dict[sub], bipartite=1)

g.add_edges_from([tuple(row) for row in df_mods[['moderator', 'subreddit']].values])
nx.write_gexf(g, '../graph.gexf')